In [31]:
import numpy as np
import netCDF4 as nc
import xarray as xr
import numpy as np
import netCDF4 as nc
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.basemap import shiftgrid
import matplotlib.pyplot as plt
import pandas as pd
from dtw import *
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from netuno import SubseriesDTW, SSTHelper

In [32]:
f_inv = lambda x: x + 180
f = lambda x: ((x+180) % 360) - 180

In [33]:
fp = '../dados/sst.mnmean.nc'
ds = SSTHelper.load_dataset(fp)
df = SSTHelper.load_dataframe(ds)

## Obtendo 5 pontos no mar
* Um ponto na região do ENSO;
 -> lat = 20°, lon = -132° (https://www.google.com.br/maps/place/20%C2%B000'00.0%22N+132%C2%B000'00.0%22W/@22.1775213,-151.1364928,3.87z/data=!4m4!3m3!8m2!3d20!4d-132?entry=ttu)

* Um ponto na costa do Chile (Corrente de Humboldt);
 -> lat = -22°, lon = -72° (https://www.google.com.br/maps/place/22%C2%B000'00.0%22S+72%C2%B000'00.0%22W/@-21.7898533,-87.474343,4.04z/data=!4m4!3m3!8m2!3d-22!4d-72?entry=ttu)

* Um ponto no meio do Mediterrâneo;
 -> lat = 36°, lon = 18° (https://www.google.com.br/maps/place/36%C2%B000'00.0%22N+18%C2%B000'00.0%22E/@36.0000043,17.9974251,17z/data=!3m1!4b1!4m4!3m3!8m2!3d36!4d18?entry=ttu)

* Um ponto no meio do Atlântico;
 -> lat = 0°, lon = -22° (https://www.google.com.br/maps/place/0%C2%B000'00.0%22N+22%C2%B000'00.0%22W/@-11.0144452,-50.6148726,3z/data=!4m3!3m2!8m1!4d-22?entry=ttu)

* Um ponto no extremo Índico (ponto de maior latitude, temperaturas mais baixas)
 -> lat = -56°, lon = 80° (https://www.google.com.br/maps/place/56%C2%B000'00.0%22S+80%C2%B000'00.0%22E/@-36.027701,54.6888129,3z/data=!4m4!3m3!8m2!3d-56!4d80?entry=ttu)

In [30]:
# enso = SubserieDTW(df, 20, f_inv(-132), 1500, 48)
split_date = '2000-01-01'

chil = SubseriesDTW(df, -22, f_inv(-72), split_date)
medi = SubseriesDTW(df, 36, f_inv(18), split_date)
atla = SubseriesDTW(df, 0, f_inv(-2), split_date)

indi = SubseriesDTW(df, -56, f_inv(-80), split_date)


Initializing
Lat: -22, Lon: 108
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/vitor/Documents/sst-time-series-ml/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_7966/3631522059.py", line 4, in <module>
    chil = SubseriesDTW(df, -22, f_inv(-72), split_date)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vitor/Documents/sst-time-series-ml/source/netuno/subseries_dtw.py", line 25, in __init__
    self.df = df
                 
NameError: name 'SSTUtil' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/vitor/Documents/sst-time-series-ml/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2142, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vitor/Documents/sst-time-series-ml/venv/lib/py